# Grid Recognition Model Training

### Import all needed modules

In [1]:
import numpy as np
import random
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import json
from types import SimpleNamespace
import seaborn; seaborn.set()
from keras.callbacks import EarlyStopping

## Building The Model

In [2]:
model = Sequential()
model.add(Conv2D(8, (3, 2), activation='relu', input_shape=(240, 320, 3)))
model.add(Conv2D(16, (3, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(16, (3, 2), activation='relu'))
model.add(Conv2D(8, (3, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(9, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 238, 319, 8)       152       
                                                                 
 conv2d_1 (Conv2D)           (None, 236, 318, 16)      784       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 118, 159, 16)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 116, 158, 16)      1552      
                                                                 
 conv2d_3 (Conv2D)           (None, 114, 157, 8)       776       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 57, 78, 8)        0         
 2D)                                                    

## Image and Config Loading

In [3]:
#Loads an image and its cfg file
def importImage(path, X, Y):
    #Load the image
    img = load_img(path)
    arr = img_to_array(img)
    
    #Load the cfg
    configFile = open(path[:-4] + '.cfg')
    configRaw = configFile.read()
    config = json.loads(configRaw, object_hook=lambda d: SimpleNamespace(**d)) 
    
    cfgArr = [config.valid, config.x1, config.y1, config.x2, config.y2, config.x3, config.y3, config.x4, config.y4]
    X.append(arr)
    Y.append(cfgArr)
    return X, Y

In [4]:
def loadAllTrainingData(data_path):
    X = []
    Y = []
    for root, dirs, files in os.walk(data_path, topdown=False):
        for f in files:
            if (f.endswith(".jpg")):
                X, Y = importImage(os.path.join(root, f), X, Y)
    return X, Y

## Training Data Splitting

In [5]:
def splitTrainingValidation(X, Y, split_size=0.2):
    if len(X) == len(Y):
        splitCount = int(len(X) * split_size)
        
        used_indices = []
        
        X_set = []
        X_val = []
        Y_set = []
        Y_val = []
        
        while len(X_val) < splitCount:
            index = random.randint(0, len(X)-1)
            if index not in used_indices:
                X_val.append(X[index])
                Y_val.append(Y[index])
                used_indices.append(index)
        
        for i in range(0, len(X)-1):
            if i not in used_indices:
                X_set.append(X[i])
                Y_set.append(Y[i])
                
        return np.array(X_set), np.array(Y_set), np.array(X_val), np.array(Y_val)

## Loading Training Data and Training

In [6]:
X, Y = loadAllTrainingData("../Traing Data Preprocessing/Training Data/Grid")
X, Y, X_val, Y_val = splitTrainingValidation(X, Y, split_size=0.2)

print(X)
print(Y)
print(X_val.shape)
print(Y_val.shape)

[[[[ 6.  6.  6.]
   [ 6.  6.  6.]
   [ 6.  6.  6.]
   ...
   [ 3.  3.  3.]
   [ 2.  2.  2.]
   [ 2.  2.  2.]]

  [[ 6.  6.  6.]
   [ 6.  6.  6.]
   [ 6.  6.  6.]
   ...
   [ 3.  3.  3.]
   [ 2.  2.  2.]
   [ 2.  2.  2.]]

  [[ 6.  6.  6.]
   [ 6.  6.  6.]
   [ 6.  6.  6.]
   ...
   [ 3.  3.  3.]
   [ 2.  2.  2.]
   [ 2.  2.  2.]]

  ...

  [[22. 22. 22.]
   [23. 23. 23.]
   [24. 24. 24.]
   ...
   [ 4.  4.  4.]
   [ 4.  4.  4.]
   [ 4.  4.  4.]]

  [[20. 20. 20.]
   [21. 21. 21.]
   [22. 22. 22.]
   ...
   [ 4.  4.  4.]
   [ 4.  4.  4.]
   [ 4.  4.  4.]]

  [[18. 18. 18.]
   [19. 19. 19.]
   [20. 20. 20.]
   ...
   [ 4.  4.  4.]
   [ 4.  4.  4.]
   [ 4.  4.  4.]]]


 [[[ 8.  8.  8.]
   [ 8.  8.  8.]
   [ 8.  8.  8.]
   ...
   [ 4.  4.  4.]
   [ 4.  4.  4.]
   [ 4.  4.  4.]]

  [[ 7.  7.  7.]
   [ 7.  7.  7.]
   [ 7.  7.  7.]
   ...
   [ 4.  4.  4.]
   [ 4.  4.  4.]
   [ 4.  4.  4.]]

  [[ 5.  5.  5.]
   [ 5.  5.  5.]
   [ 5.  5.  5.]
   ...
   [ 4.  4.  4.]
   [ 4.  4.  4.]
   [ 4.  4.

In [7]:
#es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.01)

model.fit(X, Y, epochs=50, validation_data=(X_val, Y_val), batch_size=8)

Epoch 1/50
11/11 [==============================] - 2s 172ms/step - loss: 22536.6426 - accuracy: 0.2442 - val_loss: 7991.8477 - val_accuracy: 0.1905
Epoch 2/50
11/11 [==============================] - 2s 160ms/step - loss: 4981.6812 - accuracy: 0.3721 - val_loss: 5133.9702 - val_accuracy: 0.3810
Epoch 3/50
11/11 [==============================] - 2s 158ms/step - loss: 3424.8193 - accuracy: 0.5581 - val_loss: 3650.7739 - val_accuracy: 0.5238
Epoch 4/50
11/11 [==============================] - 2s 158ms/step - loss: 2307.4138 - accuracy: 0.4767 - val_loss: 3286.5283 - val_accuracy: 0.5238
Epoch 5/50
11/11 [==============================] - 2s 158ms/step - loss: 2048.8518 - accuracy: 0.6512 - val_loss: 3172.0420 - val_accuracy: 0.5714
Epoch 6/50
11/11 [==============================] - 2s 159ms/step - loss: 1218.0107 - accuracy: 0.6163 - val_loss: 2847.6411 - val_accuracy: 0.6190
Epoch 7/50
11/11 [==============================] - 2s 159ms/step - loss: 713.5649 - accuracy: 0.5000 - val_los

### Testing Random Images

Try changing the sample number below

In [8]:
X_test = []
Y_test = []
X_test, Y_test = importImage("../Traing Data Preprocessing/Training Data/Grid/sample_98.jpg", X_test, Y_test)
X_test = np.array(X_test)
Y_predict = model.predict(X_test)
Y_predict = [[round(Y_predict[0][0]), round(Y_predict[0][1]), round(Y_predict[0][2]), round(Y_predict[0][3]), round(Y_predict[0][4]), round(Y_predict[0][5]), round(Y_predict[0][6]), round(Y_predict[0][7]), round(Y_predict[0][8])]]
print(Y_predict)
print(Y_test)

[[0, 49, 43, 306, 45, 48, 207, 310, 207]]
[[1, 41, 43, 284, 37, 45, 193, 291, 189]]
